## Data preparation with PANDAS

In [77]:
import pandas as pd


# Power plant conditions
p_conditions = pd.DataFrame({"Power plant":           ["Ahlen", "Fjället",  "Forsen",   "Kärret"],
                           "Initial reservoir level": [5800e6,  1000e6,     20e6,       13e6    ],
                           "Maximum reservoir level": [7160e6,  1675e6,     27e6,       13e6    ],
                           "Minimum reservoir level": [5800e6,  1000e6,     10e6,       6e6     ],
                           "Discharge capacity":      [540,     135,        975,        680     ],
                           "Power conversion":        [0.52,    1.17,       0.29,       0.05    ],
                           "Maximum spillage":        [820,     930,        360,        400     ],
                           "Local flow":              [177,     28,         8,          29      ],
                          })

# Time conditions
t_conditions = pd.DataFrame({"Time": range(1,12),
                             "Cost": [45, 55, 95, 80, 140, 150, 80, 70, 130, 0, 0]})

# Flow conditions (Connections between the power plants)
f_conditions = pd.DataFrame({"From": ["Ahlen",  "Fjället",  "Forsen"],
                             "To":   ["Forsen", "Forsen",   "Kärret"],
                             "Time": [2,        1,          1]
                            })


## GAMSPy FTW

### Initializing all our Parameters and Varaibales etc

In [78]:

from gamspy import Container, Set, Variable, Parameter, Equation, Sum, Model, Sense, Alias


m = Container()

t = Set(m, name="t", description="time in hours", records=t_conditions['Time']) # time at begining of hour 1, 2, 3, ...
p = Set(m, name="p", description="Power plant", records=p_conditions['Power plant'])

# Create alias for set p
p_up = Alias(m, name="p_up", alias_with=p)

# Parameter and Variables definitions


delay = Parameter(m, name="delay", domain=[p,p], description="Time delay for upstream plants", records=f_conditions[['From', 'To', 'Time']])

prices = Parameter(m, name="prices", domain=t, description="Prices (MWh) at different hours", records=t_conditions[['Time', 'Cost']])

reservoir_init = Parameter(m, name="reservoir_init", domain=p, description="Initial reservoir level", records=p_conditions[['Power plant', 'Initial reservoir level']])
reservoir_max = Parameter(m, name="reservoir_max", domain=p, description="Maximum reservoir level", records=p_conditions[['Power plant', 'Maximum reservoir level']])
reservoir_min = Parameter(m, name="reservoir_min", domain=p, description="Minimum reservoir level", records=p_conditions[['Power plant', 'Minimum reservoir level']])
discharge_max = Parameter(m, name="discharge_max", domain=p, description="Discharge capacity", records=p_conditions[['Power plant', 'Discharge capacity']])
power_conversion = Parameter(m, name="power_conversion", domain=p, description="Power conversion", records=p_conditions[['Power plant', 'Power conversion']])
spillage_max = Parameter(m, name="spillage_max", domain=p, description="Maximum spillage", records=p_conditions[['Power plant', 'Maximum spillage']])
local_flow = Parameter(m, name="local_flow", domain=p, description="Local flow", records=p_conditions[['Power plant', 'Local flow']])

discharge = Variable(m, name="discharge", type="positive", domain=[t,p], description="Discharge rate at each power plant at each time")
spillage = Variable(m, name="spillage", type="positive", domain=[t,p], description="Spillage rate at each power plant at each time")
reservoir_level = Variable(m, name="reservoir_level", domain=[t,p], description="Reservoir level at each power plant at each time")
potential_volume = Variable(m, name="potential_volume", domain=p, description="Potential volume at each power plant at last time (T=10)")

### Equations and condtions

In [79]:
# Discharge criteria
discharge.up[t,p] = discharge_max[p]

# Spillage criteria
spillage.up[t,p] = spillage_max[p]

# Reservoir level criteria
reservoir_level.lo[t,p].where[t.ord > 1] = reservoir_min[p]
reservoir_level.up[t,p].where[t.ord > 1] = reservoir_max[p]
reservoir_level.fx[t,p].where[t.first] = reservoir_init[p] # Initial reservoir level should be set to reservoir initial level

# Potential volume criteria
potential = Equation(m, name="potential", domain=p, description="Potential volume at each power plant based on the volume at last time (T=10) and power plant above")
potential[p] = potential_volume[p] == Sum(p_up.where[delay[p_up,p]>0], potential_volume[p_up]) + reservoir_level["10",p] - reservoir_min[p]

# Single reservoir equation for all plants
reservoirs = Equation(m, name="reservoirs", domain=[t,p], description="Reservoir level at power plant (p) at different hours (t)")
reservoirs[t,p].where[t.ord > 1] = reservoir_level[t,p] == reservoir_level[t.lag(1),p] + 3600 * (
        # Upstream inflows
        Sum(p_up.where[delay[p_up,p]>0], 
            discharge[t.lag(delay[p_up, p]), p_up] + spillage[t.lag(delay[p_up, p]), p_up]
        )

        # Local inflow
        + local_flow[p]

        # Outflows
        - discharge[t.lag(1),p]
        - spillage[t.lag(1),p]
    )

### Obejctive

In [80]:
obj = Sum((t, p), prices[t]*3600*power_conversion[p]*discharge[t,p]) + Sum(p, 45*power_conversion[p]*potential_volume[p])

## Solution

In [81]:
flow = Model(m, name="flow", equations=m.getEquations(), objective=obj, problem="LP", sense=Sense.MAX)
flow.solve(solver="CPLEX")

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,1220105320.00005,45,129,LP,CPLEX,0.016


1406579020	

In [82]:
from IPython.display import HTML

def horizontal(dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 32px">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))

### Primals Discharge, Spillage, Reservoir Level

In [83]:
# Convert levels to pandas DataFrames for easier handling
discharge_primal_df = pd.DataFrame(discharge.l.records).pivot(index='t', columns='p', values='level')
spillage_primal_df = pd.DataFrame(spillage.l.records).pivot(index='t', columns='p', values='level')
reservoir_level_primal_df = pd.DataFrame(reservoir_level.l.records).pivot(index='t', columns='p', values='level')

horizontal([discharge_primal_df, spillage_primal_df, reservoir_level_primal_df])

p,Ahlen,Fjället,Forsen,Kärret
t,,,,
1,0.0,0.0,0.000000,29.0
2,0.0,0.0,0.000000,680.0
3,0.0,0.0,975.000000,680.0
4,0.0,0.0,0.000000,680.0
5,522.0,33.0,975.000000,680.0
6,540.0,135.0,975.000000,680.0
7,0.0,0.0,263.777778,680.0
8,0.0,0.0,0.000000,680.0
9,531.0,84.0,975.000000,680.0


### Duals Discharge, Spillage, Reservoir Level

In [84]:
# Convert marginals to pandas DataFrames for easier handling
discharge_duals_df = pd.DataFrame(discharge.m.records).pivot(index='t', columns='p', values='marginal')
spillage_duals_df = pd.DataFrame(spillage.m.records).pivot(index='t', columns='p', values='marginal')
reservoir_level_duals_df = pd.DataFrame(reservoir_level.m.records).pivot(index='t', columns='p', values='marginal')

horizontal([discharge_duals_df, spillage_duals_df, reservoir_level_duals_df])



p,Ahlen,Fjället,Forsen,Kärret
t,,,,
1,-177840.0,-400140.0,-36540.0,0.0
2,-159120.0,-358020.0,-26100.0,1800.0
3,-84240.0,-189540.0,15660.0,9000.0
4,-112320.0,-252720.0,-0.0,6300.0
5,0.0,0.0,62640.0,17100.0
6,18720.0,42120.0,73080.0,18900.0
7,-1980.0,-210600.0,0.0,6300.0
8,-20700.0,-252720.0,-10440.0,4500.0
9,0.0,0.0,52200.0,15300.0


## Extra

In [86]:
potential_volume.records

,p,level,marginal,lower,upper,scale
0,Ahlen,0.0,0.0,-inf,inf,1.0
1,Fjället,0.0,0.0,-inf,inf,1.0
2,Forsen,0.0,0.0,-inf,inf,1.0
3,Kärret,3240800.0,0.0,-inf,inf,1.0
